<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/8_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Rule 1

單一點落在3外

#### 1.1 一般 (未分群，只查看 value)

In [ ]:
import pandas as pd

def check_western_electric_rule1(df, column_name, k):
    """
    檢查 Western Electric Rule 1 是否被觸發。

    參數:
    df (pandas.DataFrame): 包含過程數據的 DataFrame。
    column_name (str): DataFrame 中包含過程變數的列名。
    k (float): 控制限的倍數，通常為 3 倍標準差。

    返回:
    violations (pandas.DataFrame): 包含違規點的 DataFrame。
    """
    # 計算平均值和標準差
    mean = df[column_name].mean()
    std_dev = df[column_name].std()

    # 計算控制限
    upper_control_limit = mean + k * std_dev
    lower_control_limit = mean - k * std_dev

    # 檢測違規的點
    mask = (df[column_name] > upper_control_limit) | (df[column_name] < lower_control_limit)
    violations = df[mask]

    return violations

# 示例使用
# 假設 'process_data' 是一個 DataFrame，並且我們關心名為 'measurement' 的列
# process_data = pd.read_excel('path_to_your_file.xlsx')  # 加載你的數據
# rule1_violations = check_western_electric_rule1(process_data, 'measurement', 3)
# print(rule1_violations)


#### 1.2 有分群

## Rule 2

連續9點同一側

In [ ]:
import pandas as pd

def western_electric_rule2(df, k, col_name):
    # 初始化一個空的 DataFrame 來存儲結果
    columns = ['col1', 'col2', 'col_value'] + [f'point{i}' for i in range(k, 0, -1)] + [f'point{i}_value' for i in range(k, 0, -1)]
    results = pd.DataFrame(columns=columns)

    # 檢查每個點是否符合連續 k 點在中心線同一側的規則
    for i in range(len(df) - k + 1):
        sub_df = df[col_name].iloc[i:i+k]
        if all(sub_df > df[col_name].mean()) or all(sub_df < df[col_name].mean()):
            # 如果找到符合條件的連續 k 點，記錄這些點及其索引
            row = {
                'col1': 'Sample',  # 根據實際需求自行調整
                'col2': 'Sample',  # 根據實際需求自行調整
                'col_value': df[col_name].iloc[i+k-1]
            }
            row.update({f'point{j}': i+k-j for j in range(k)})
            row.update({f'point{j}_value': df[col_name].iloc[i+k-j] for j in range(k)})
            results = results.append(row, ignore_index=True)

    return results

# 假設的 DataFrame 和用例
data = {
    'Measurement': [1, 2, 3, 4, 5, 4, 3, 2, 1, 2, 3, 4, 5, 6, 5, 4, 3, 2, 1]
}
df = pd.DataFrame(data)

# 調用函數
output_df = western_electric_rule2(df, 3, 'Measurement')
print(output_df)


In [ ]:
def western_electric_rule2_updated(df, k, col_name):
    # 初始化一個空的 DataFrame 來存儲結果
    columns = ['col1', 'col2']
    for j in range(k, 0, -1):
        columns.append(f'point{j}')
        columns.append(f'point{j}_value')

    results = pd.DataFrame(columns=columns)

    # 檢查每個點是否符合連續 k 點在中心線同一側的規則
    for i in range(len(df) - k + 1):
        sub_df = df[col_name].iloc[i:i+k]
        if all(sub_df > df[col_name].mean()) or all(sub_df < df[col_name].mean()):
            # 如果找到符合條件的連續 k 點，記錄這些點及其索引
            row = {
                'col1': df.iloc[i+k-1]['col1'],  # 根據實際需求自行調整
                'col2': df.iloc[i+k-1]['col2'],  # 根據實際需求自行調整
            }
            for j in range(k, 0, -1):
                row[f'point{j}'] = i+k-j
                row[f'point{j}_value'] = df[col_name].iloc[i+k-j]

            results = results.append(row, ignore_index=True)

    return results

# 測試函數
result_updated = western_electric_rule2_updated(df, 9, 'col_name')
import ace_tools as tools; tools.display_dataframe_to_user(name="Updated Western Electric Rule Results", dataframe=result_updated)


## Rule 3 (已做過)

## Rule 4

In [ ]:
def western_electric_rule4_v2(df, k, col_name):
    # 初始化一個空的 DataFrame 來存儲結果
    columns = ['col1', 'col2', 'col_value']
    for i in range(k, 0, -1):
        columns.append(f'point{i}')
        columns.append(f'point{i}_value')

    results = pd.DataFrame(columns=columns)

    # 檢查每個點是否符合連續 k 點交替上升下降的規則
    for i in range(len(df) - k + 1):
        sub_df = df[col_name].iloc[i:i+k].values
        # 檢查是否交替上升下降
        is_alternating = all((sub_df[j] > sub_df[j-1] and sub_df[j+1] < sub_df[j]) or
                             (sub_df[j] < sub_df[j-1] and sub_df[j+1] > sub_df[j])
                             for j in range(1, k-1))
        if is_alternating:
            # 如果找到符合條件的連續 k 點，記錄這些點及其索引
            row = {
                'col1': df.iloc[i+k-1]['col1'],  # 根據實際需求自行調整
                'col2': df.iloc[i+k-1]['col2'],  # 根據實際需求自行調整
                'col_value': df[col_name].iloc[i+k-1]
            }
            for j in range(k):
                row[f'point{k-j}'] = i+k-j
                row[f'point{k-j}_value'] = df[col_name].iloc[i+k-j]

            results = results.append(row, ignore_index=True)

    return results

# 測試函數
result_rule4_v2 = western_electric_rule4_v2(df, 14, 'col_name')
tools.display_dataframe_to_user(name="Western Electric Rule 4 Results (Corrected)", dataframe=result_rule4_v2)


* (PPTX)補充

1. https://www.ignsw.com/%E8%BE%A6%E5%85%AC%E5%AE%A4%E8%87%AA%E5%8B%95%E5%8C%96%E4%B9%8Bpowerpoint%E6%93%8D%E4%BD%9C/

2. https://medium.com/%E6%9F%BF%E7%94%9C%E8%8C%B6%E9%A6%99/python-%E8%A3%BD%E4%BD%9C-ppt-%E4%B8%8A-accb8f5bb461

3. https://www.dszhp.com/python-pptx.html

4. https://ithelp.ithome.com.tw/articles/10248794?sc=iThomeR